## Import required libraries and packages

In [1]:
import os
import yaml

import pandas as pd
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate


from prompt import Prompt

## Helper code to extract paths, sheet state

In [2]:
config_file = "./config/config.yaml"
agent_name = "gpt-4o-mini"
model_name = "gpt-4o-mini"
few_shot_count = 2

In [3]:
def set_agent_configuration(configuration_file_path, agent_name):
    with open(configuration_file_path, mode="r") as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]
    return agent_config

In [4]:
agent_config = set_agent_configuration(
    configuration_file_path=config_file, agent_name=agent_name
)
prompt = Prompt(
    agent_configuration=agent_config,
    agent_name=agent_name,
    model_name=model_name,
    few_shot_count=few_shot_count,
)

In [5]:
def load_html_documents():
    excel_js_api_docs = [
        str(path) for path in prompt.create_path_generator("../excel_js_api_docs")
    ]
    documents = []
    for filename in excel_js_api_docs:
        html_loader = UnstructuredHTMLLoader(file_path=filename)
        document = html_loader.load()
        documents.append(document[0])
    return documents

In [6]:
def split_documents_into_chunks(documents, embedding_model):
    semantic_splitter = SemanticChunker(
        embeddings=embedding_model,
        breakpoint_threshold_type="gradient",
        breakpoint_threshold_amount=0.8,
    )
    chunks = semantic_splitter.split_documents(documents)
    return chunks

In [7]:
def embed_chunks_and_save_vector_database(chunks, database_directory, embedding_model):
    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_model,
        persist_directory=database_directory,
    )
    return vector_store

In [8]:
vector_database_directory = "../vector_db/"
embedding_model = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-small"
)
if not os.path.exists(f"{vector_database_directory}/chroma.sqlite3"):
    print("Vector Database does not exist. Creating a new database ...")
    html_documents = load_html_documents()
    splitted_chunks = split_documents_into_chunks(html_documents, embedding_model)
    vector_database = embed_chunks_and_save_vector_database(
        splitted_chunks, vector_database_directory, embedding_model
    )
else:
    print("Vector database already exists. Loading the database ...")
    vector_database = Chroma(
        persist_directory=vector_database_directory, embedding_function=embedding_model
    )
    vector_database.get()

Vector database already exists. Loading the database ...


## Retriever

In [9]:
retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

## Get sheet state

In [10]:
sheet_state = prompt.get_sheet_state("../JS_Excel_files/EntireShippingCosts.xlsx")

## Get JavaScript code answers

In [11]:
def read_js_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [12]:
task_10_js_code = read_js_file("../JS_Excel_files/task_10.js")
task_11_js_code = read_js_file("../JS_Excel_files/task_11.js")

## Few shot prompt

In [13]:
few_shot_examples = [
    {
        "input": f"Duplicate the first 10 rows in Sheet1 in Sheet2. \n Sheet state: {sheet_state}", 
        "output": f"{task_10_js_code}"
    },
    {
        "input": f"Display only the rows where the distances to Milwaukee are less than 2000 miles. \n Sheet state: {sheet_state}", 
        "output": f"{task_11_js_code}"
    },
]

In [14]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("ai", "{output}"),
])

In [15]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_shot_examples,
)

## Final Prompt

In [16]:
user_input = (
    "\n"
    "excel_js_api: {excel_js_api}\n"
    "input: {input}\n"
)

In [17]:
system_message = (
    "You are a JavaScript, TypeScript, and Excel expert.\n"
    "Generate the TypeScript code to manipulate Excel using Excel JavaScript API for each given task.\n"
    "The Excel sheet state is provided.\n"
)

In [18]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        few_shot_prompt,
        ("human", user_input)
    ]
)

In [19]:
with open(config_file, mode="r") as file:
    config = yaml.load(file, Loader=yaml.Loader)
task_df = pd.read_excel(config['path']['task_path'], header=0)

for index, row in task_df.iloc[:2].iterrows():
    source_path = os.path.join(config['path']['source_path'], row['Sheet Name']+'.xlsx')
    input_task = (
        f"Task: {row['Instructions']}\n"
        f"Context: {row['Context']}\n"
        f"Sheet State: {prompt.get_sheet_state(source_path)}"
    )
    print(final_prompt.format(input=input_task, excel_js_api="HTML_docs"))
    

System: You are a JavaScript, TypeScript, and Excel expert.
Generate the TypeScript code to manipulate Excel using Excel JavaScript API for each given task.
The Excel sheet state is provided.

Human: Duplicate the first 10 rows in Sheet1 in Sheet2. 
 Sheet state: Sheet state: Sheet "Sheet1" has 5 columns (Headers are A: "Customers", B: "Seattle", C: "Milwaukee", D: "Birmingham", E: "Oakland") and 71 rows (1 header row and 70 data rows).
AI: $("#run").on("click", () => tryCatch(run));

async function run() {
  await Excel.run(async (context) => {
    let sheets = context.workbook.worksheets;
    let sheet1 = sheets.getItem("Sheet1");

    sheet1.load("position"); // Load the position of Sheet1

    await context.sync(); // Sync before using loaded properties

    // Add a new sheet
    let sheet2 = sheets.add("Sheet2");
    sheet2.position = sheet1.position; // Insert before Sheet1

    // Copy headers
    let headers = sheet1.getRange("A1:E1");
    sheet2.getRange("A1:E1").copyFrom(hea